In [55]:
import importlib
from CASutils import lensread_utils as lens
from CASutils import readdata_utils as read
from CASutils import calendar_utils as cal
from glob import glob
import pandas as pd
import numpy as np
import xarray as xr
import warnings
import sys

importlib.reload(lens)
importlib.reload(read)
importlib.reload(cal)

warnings.filterwarnings('ignore')


Set up member names and file locations

In [56]:
nmems=50
memstr = lens.lens2memnamegen(nmems)
preccdir="/project/mojave/cesm2/LENS/atm/month_1/PRECC/"
precldir="/project/mojave/cesm2/LENS/atm/month_1/PRECL/"

Set up dates required

In [57]:
ybeg=1979 ; mbeg = 1 ; yend=2014 ; mend=12
nmonths=(yend-ybeg-1)*12 + (12-mbeg+1) + mend
datebeg = str(ybeg)+'-'+str(mbeg).zfill(2)
dateend = str(yend)+'-'+str(mend).zfill(2)

In [58]:
for imem in range(0,len(memstr),1):
    
    print(imem)
    
    # ----read in PRECC
    fhist=preccdir+"b.e21.BHISTcmip6.f09_g17.LE2-"+memstr[imem]+".cam.h0.PRECC.*.nc"
    precchist = read.read_sfc_cesm(fhist,datebeg,dateend)
    
    if (precchist.time.size != nmonths):
        print("something's wrong.  nmonths="+str(nmonths)+" but got "+str(precchist.time.size))
        sys.exit("Failed at PRECC for member "+memstr[imem])
    # ------------------
        
    # ----read in PRECL
    fhist=precldir+"b.e21.BHISTcmip6.f09_g17.LE2-"+memstr[imem]+".cam.h0.PRECL.*.nc"
    preclhist = read.read_sfc_cesm(fhist,"1979-01","2014-12")
    
    if (preclhist.time.size != nmonths):
        print("something's wrong.  nmonths="+str(nmonths)+" but got "+str(preclhist.time.size))
        sys.exit("Failed at PRECL for member "+memstr[imem])
    # ------------------
   
    precthist=precchist['PRECC'] + preclhist['PRECL']
    precthist = precthist.rename('PRECT')
    precthist = precthist.to_dataset()
    
    prectdjf = cal.season_ts(precthist,'PRECT','DJF')
    prectmam = cal.season_ts(precthist,'PRECT','MAM')
    prectjja = cal.season_ts(precthist,'PRECT','JJA')
    prectson = cal.season_ts(precthist,'PRECT','SON')
    
    if (imem == 0):
        djfgather = xr.DataArray(np.zeros([nmems, prectdjf.time.size, prectdjf.lat.size, prectdjf.lon.size]),
                                coords=[memstr, prectdjf['time'], prectdjf['lat'], prectdjf['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTDJF')
        
        mamgather = xr.DataArray(np.zeros([nmems, prectmam.time.size, prectmam.lat.size, prectmam.lon.size]),
                                coords=[memstr, prectmam['time'], prectmam['lat'], prectmam['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTMAM')
        
        jjagather = xr.DataArray(np.zeros([nmems, prectjja.time.size, prectjja.lat.size, prectjja.lon.size]),
                                coords=[memstr, prectjja['time'], prectjja['lat'], prectjja['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTJJA')
        
        songather = xr.DataArray(np.zeros([nmems, prectson.time.size, prectson.lat.size, prectson.lon.size]),
                                coords=[memstr, prectson['time'], prectson['lat'], prectson['lon']],
                                dims = ['Member','time','lat','lon'], name='PRECTSON')
        
    djfgather[imem,:,:,:] = prectdjf*86400.*1000.
    mamgather[imem,:,:,:] = prectmam*86400.*1000.
    jjagather[imem,:,:,:] = prectjja*86400.*1000.
    songather[imem,:,:,:] = prectson*86400.*1000.    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [59]:
outdir="/project/cas/islas/savs/drought/LENS2_vs_LENS1/outputdata/seasonalaverages/LENS2/"

djfgather.to_netcdf(path=outdir+"pr_djf.nc")
mamgather.to_netcdf(path=outdir+"pr_mam.nc")
jjagather.to_netcdf(path=outdir+"pr_jja.nc")
songather.to_netcdf(path=outdir+"pr_son.nc")